In [13]:
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re
import pandas as pd
from datetime import datetime

In [19]:
def getcomments(comment,token= os.getenv("TOKEN")):
    headers = {"Authorization": f"token {token}"}
    res=requests.get(f'https://api.github.com/repos/ironhack-datalabs/datamad0820/issues/{comment}/comments',headers=headers)
    data=res.json()
    return data
def regex(x):
    return re.findall('@\w*-?\w+', x)
def image(x):
    return re.sub(re.findall('\((.*?)\)',x))
def cleantitle(x):
    return ((str(re.findall('\[(.*?)\]',x)).split('['))[1].split("'"))[1]
def instructorcomment(comment):
    try:
        return re.findall('@\w*-?\w+',comment[0]['body'])
    except:
        return None
    
def https(comment):
    try:
        try:
            z = re.findall(r'https:.*jpg|.*png|.*jpeg',comment[0]['body'])
            z = str(z).split('(')
            z = z[1].split("'")
            return z[0]
        except: 
            z = re.findall(r'https:.*jpg|.*png|.*jpeg',comment[0]['body'])
            z = str(z).split('(')
            return z[0]
    except:
        return None 
def instructor(comment):
    try:
        return comment[0]['user']['login']
    except:
        return None
def putasnotas(comment):
    try:
        z= re.findall(r'grade:.*-',comment[0]['body'])
        z = str(z).split(':')
        z = z[1].split("-")
        return z[0]
    except:
        return None
def lab(data):
    try:
        return re.findall(r'\[(.*?)\]',data)[0]
    except: return None
    
def getpull (x,token= os.getenv("TOKEN"), query_params = {}):
        headers = {
            "Authorization": f"token {token}"}
        res = requests.get(f'https://api.github.com/repos/ironhack-datalabs/datamad0820/pulls/{x}', headers=headers)

        data = res.json()
        comment=getcomments(x)
        if res.status_code == 404:
            return 'exit'
        else:
            return {
                #"Title":cleantitle(data['title']),
                "Id":data['number'],
                "Lab":lab(data['title']),
                "State":data['state'],
                "User":data['user']['login'],
                "Creado":data['created_at'],
                "Closed":data['closed_at'],
                "Comentario":data['body'],
                "Instructor":instructor(comment),
                "InstructorComment":instructorcomment(comment),
                "InstructorCommentImage":https(comment),
                "InstructorGrades":putasnotas(comment)
        }
            

In [20]:
data=[]
for i in range(1,600):
    if getpull(i) == 'exit':
        pass
    else:
        data.append(getpull(i))


In [21]:
data

[{'Id': 1,
  'Lab': 'lab-resolving-conflicts',
  'State': 'closed',
  'User': 'miguelgimenezgimenez',
  'Creado': '2020-08-17T13:53:33Z',
  'Closed': '2020-08-17T14:07:51Z',
  'Comentario': 'Hay unos cuantos commits que al principio me ha hecho fast forward ...',
  'Instructor': None,
  'InstructorComment': None,
  'InstructorCommentImage': None,
  'InstructorGrades': None},
 {'Id': 2,
  'Lab': 'lab-resolving-git-conflicts',
  'State': 'closed',
  'User': 'miguelgimenezgimenez',
  'Creado': '2020-08-17T14:10:42Z',
  'Closed': '2020-08-18T09:10:51Z',
  'Comentario': 'Hay unos cuantos commits que al principio me ha hecho fast forward ...',
  'Instructor': 'agalvezcorell',
  'InstructorComment': [],
  'InstructorCommentImage': 'https://user-images.githubusercontent.com/57899051/90494122-7332f800-e143-11ea-8527-f257dcbb6101.jpeg',
  'InstructorGrades': '10'},
 {'Id': 3,
  'Lab': 'lab-resolving-git-conflicts',
  'State': 'closed',
  'User': 'IreneLopezLujan',
  'Creado': '2020-08-17T14:13:3

In [22]:
proyectito = json.dumps(data)

In [23]:
jsn = pd.DataFrame(data)

In [24]:
jsn.to_json('output/pull.json',orient="records")